In [1]:
import cv2
import numpy as np
from PIL import Image
import pandas
import math
from generate_inputs import GenerateInputVector
from generate_inputs import device
from generate_inputs import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [2]:
dir = './data/traj_2/'
pose_list, image_list, image1_list, image2_list, rel_pose_list = GenerateInputVector(dir)

In [3]:
from torch.utils.data import Dataset, DataLoader



# Verifying input dimensions
class Net(nn.Module):
    #Architecture is LeNet modification
    def __init__(self):
        super(Net, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(2, 16, kernel_size=5, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(), 
            nn.Conv2d(128, 256, kernel_size=5, stride=2, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(),)
        self.linear_layers = nn.Sequential(
            nn.Linear(4096, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            )

    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, 1)
        x = self.linear_layers(x)
        return x[0][0]


device = torch.device("cuda")
net = Net()
net = net.to(device)
print(net)

input = torch.randn(1, 2, 224, 224).to(device)
out = net(input)
print(out.shape)

x = np.stack([image1_list[0], image2_list[0]])
x = np.expand_dims(x, axis=0)
print(x.shape)

Net(
  (conv_layers): Sequential(
    (0): Conv2d(2, 16, kernel_size=(5, 5), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_feat

In [4]:
import random
import torch.optim as optim
idxs = list(range(len(image1_list)))
#random.shuffle(idxs)

net_x = Net()
net_x = net_x.to(device)

optimizer = optim.Adam(net_x.parameters(), lr=0.0001)

#random.shuffle(idxs)
train_idxs = idxs[:int(len(image1_list) * 0.8)]
test_idxs = idxs[int(len(image1_list) * 0.8):]
min_loss = 1

for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(rel_pose_list[idx][0]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_x, "net_x_2.pyt")
        min_loss = test_total / len(test_idxs)
    for idx in train_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(rel_pose_list[idx][0]).float().to(device)
        #print(out.shape, target.shape)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        optimizer.zero_grad()
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))

Test loss: 0.3836405336922534
New lowest test loss, saved!
Train loss: 0.05504012868248121
Test loss: 0.14367590617445966
New lowest test loss, saved!
Train loss: 0.04047423058442282
Test loss: 0.11223090778349254
New lowest test loss, saved!
Train loss: 0.0379344704450135
Test loss: 0.10816981263518946
New lowest test loss, saved!
Train loss: 0.03482230418491347
Test loss: 0.1009101125731254
New lowest test loss, saved!
Train loss: 0.033453806534710695
Test loss: 0.0818525785436576
New lowest test loss, saved!
Train loss: 0.02656361621487535
Test loss: 0.08285717725819476
Train loss: 0.022237277243989675
Test loss: 0.08304237977250546
Train loss: 0.018954284762006234
Test loss: 0.09098473217469707
Train loss: 0.017573184861990224
Test loss: 0.07900265548966721
New lowest test loss, saved!
Train loss: 0.01639602768483461


In [5]:
# Training X- network

import random
import torch.optim as optim
idxs = list(range(len(image1_list)))
random.shuffle(idxs)
net_x = Net()
net_x = net_x.to(device)
optimizer = optim.Adam(net_x.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(image1_list) * 0.8)]
test_idxs = idxs[int(len(image1_list) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(rel_pose_list[idx][0]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_x, "net_x_2.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(rel_pose_list[idx][0]).float().to(device)
        #print(out.shape, target.shape)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        optimizer.zero_grad()
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.2539747405686388
New lowest test loss, saved!
Train loss: 0.2553569317493737
Test loss: 0.2539747405686388


KeyboardInterrupt: 

In [6]:
# Training Y- network

import random
import torch.optim as optim
idxs = list(range(len(image1_list)))
random.shuffle(idxs)
net_y = Net()
optimizer = optim.Adam(net_y.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(image1_list) * 0.8)]
test_idxs = idxs[int(len(image1_list) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(rel_pose_list[idx][1]).float().cuda()
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_y, "net_y_2.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(rel_pose_list[idx][1]).float().cuda()
        #criterion = nn.MSELoss()
        optimizer.zero_grad()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


/home/snarasimhan/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Test loss: 0.06934934796342554
New lowest test loss, saved!
Train loss: 0.06934462997396713
Test loss: 0.056546016219212326
New lowest test loss, saved!
Train loss: 0.05649986668777784
Test loss: 0.005142409493488073
New lowest test loss, saved!
Train loss: 0.005295995845778969
Test loss: 0.056623167368603226
Train loss: 0.0566025965206657
Test loss: 0.04293317949532105
Train loss: 0.042929369274511436
Test loss: 0.00805713707449527
Train loss: 0.008079838621347868
Test loss: 0.028063110262018673
Train loss: 0.028025350178265494
Test loss: 0.03630810165415127
Train loss: 0.03625017521397258
Test loss: 0.02779315715049401
Train loss: 0.027752790297349495
Test loss: 0.011516068785599845
Train loss: 0.011595740350759873


In [5]:
# Training Angle- network

import random
import torch.optim as optim
idxs = list(range(len(image1_list)))
random.shuffle(idxs)
net_angle = Net()
optimizer = optim.Adam(net_angle.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(image1_list) * 0.8)]
test_idxs = idxs[int(len(image1_list) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(rel_pose_list[idx][2]).float().cuda()
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_angle, "net_angle_2.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([image1_list[idx], image2_list[idx]])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(rel_pose_list[idx][2]).float().cuda()
        optimizer.zero_grad()
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.0018061496475784789
New lowest test loss, saved!
Train loss: 0.0017800304391827136
Test loss: 0.0045937059533164406
Train loss: 0.004494457818568011
Test loss: 0.0012841282956385603
New lowest test loss, saved!
Train loss: 0.001224031749492032
Test loss: 0.001449465198994546
Train loss: 0.0014238361287837323
Test loss: 0.0015872785368824297
Train loss: 0.0015678407576588272
Test loss: 0.0012459499932667745
New lowest test loss, saved!
Train loss: 0.0012187522431093111
Test loss: 0.0009906131112756272
New lowest test loss, saved!
Train loss: 0.0009516366723274594
Test loss: 0.0009798973948516134
New lowest test loss, saved!


KeyboardInterrupt: 